# Материалы

Материалы: [Дашборд](https://public.tableau.com/app/profile/savva.savinov/viz/FinalProjYandex/Dashboard1)

# Поиск клиентов с неоптимальными тарифами

 Поступил заказ от коммерческого директора телеком провайдера «Нупозвони», он предоставил нам датасет по клиентам (колл-центрам) и тарифным планам. В городе появился новый конкурент.Необходимо проанализировать данные и выяснить как смена тарифного плана для клиента влияет на выручку компании «Нупозвони». Были сформулированы задачи и общая цель.

Цель - на основе данных за 3 месяца исследовать как измениться выручка, если все активные клиенты, которые неэффективно используют свои тарифные планы, будут использовать оптимальные тарифные планы

Задачи:
+ Определить активных клиентов (параметры: количество звонков; отношение пропущенных звонков к общему количеству не превышает 70%; среднее количество времени разговора на оператора по каждому месяцу)

+ Выяснить сколько клиентов переплачивают и как измениться выручка в сравнении с двумя другими тарифными планами.

+ Найти более выгодный (оптимальный) тарифный план активным клиентам

+ Рассчитать текущую выручку

+ Рассчитать выручку после перевода активных клиентов на оптимальные тарифы

+ Протестировать гипотезы

+ Подготовить презентацию с графиками по ключевым метрикам (динамика выручки по месяцам на текущий момент, а также при переводе на другой тариф)

+ Подготовить дашборд



В нашем распоряжении 2 датасета

В первом отражена информация по id клиентов и id их операторов, характер звонков (внешний/внутренний, входящий/исходящий), длительность звонков, дата совершения звонка.

Во втором представлена информация каким из трех тарифных планов пользуется каждый клиет и дата начала сотрудничества

# Содержание

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Подготовка данных
### Загрузка библиотек и открытие датасетов

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as st
from datetime import datetime
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
tl = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Source/telecom_dataset.csv')
tf = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Source/telecom_clients.csv')

In [5]:
tl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              53902 non-null  int64  
 1   date                 53902 non-null  object 
 2   direction            53902 non-null  object 
 3   internal             53785 non-null  object 
 4   operator_id          45730 non-null  float64
 5   is_missed_call       53902 non-null  bool   
 6   calls_count          53902 non-null  int64  
 7   call_duration        53902 non-null  int64  
 8   total_call_duration  53902 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 3.3+ MB


In [6]:
tl.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
1,166377,2019-08-05 00:00:00+03:00,out,True,880022.0,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022.0,True,3,0,25


In [7]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      732 non-null    int64 
 1   tariff_plan  732 non-null    object
 2   date_start   732 non-null    object
dtypes: int64(1), object(2)
memory usage: 17.3+ KB


In [8]:
tf.head()

,user_id,tariff_plan,date_start
0,166713,A,2019-08-15
1,166901,A,2019-08-23
2,168527,A,2019-10-29
3,167097,A,2019-09-01
4,168193,A,2019-10-16


In [9]:
tl=tl.merge(tf, how='left')

### Предобработка данных
#### Работа с типами данных

In [10]:
tl['date'] = pd.to_datetime(tl['date'], format ='%Y-%m-%d').dt.date
tl['date'] = tl['date'].astype("datetime64[ns]")
tl['operator_id'] = tl['operator_id'].astype('Int64')
tl['month'] = pd.to_datetime(tl['date']).dt.month
#tl['date'] = pd.to_datetime(tl['date'].dt.date)

Количество колл-центров

In [11]:
tl['user_id'].nunique()

307

Количество операторов

In [12]:
tl['operator_id'].nunique()

1092

#### Работа с дубликатами

In [13]:
tl.duplicated().sum()

4900

In [14]:
tl=tl.drop_duplicates().reset_index(drop=True)

Удлаляем явные дубликаты

#### Работа с  пропусками

In [15]:
tl.isna().sum()

user_id                   0
date                      0
direction                 0
internal                110
operator_id            7456
is_missed_call            0
calls_count               0
call_duration             0
total_call_duration       0
tariff_plan               0
date_start                0
month                     0
dtype: int64

In [16]:
tl.isna().mean()

user_id                0.000000
date                   0.000000
direction              0.000000
internal               0.002245
operator_id            0.152157
is_missed_call         0.000000
calls_count            0.000000
call_duration          0.000000
total_call_duration    0.000000
tariff_plan            0.000000
date_start             0.000000
month                  0.000000
dtype: float64

In [17]:
tl.dropna(subset =['internal'], inplace= True)

0.002 процента от всех данных, можем удалить данные строки, потому что по `internal` нужны точные данные для определения выручки

In [18]:
tl.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration,tariff_plan,date_start,month
0,166377,2019-08-04,in,False,<NA>,True,2,0,4,B,2019-08-01,8
1,166377,2019-08-05,out,True,880022,True,3,0,5,B,2019-08-01,8
2,166377,2019-08-05,out,True,880020,True,1,0,1,B,2019-08-01,8
3,166377,2019-08-05,out,True,880020,False,1,10,18,B,2019-08-01,8
4,166377,2019-08-05,out,False,880022,True,3,0,25,B,2019-08-01,8


In [19]:
tl.groupby('internal').agg({'internal':'count'})

,internal
internal,
False,43239
True,5653


In [20]:
tl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48892 entries, 0 to 49001
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              48892 non-null  int64         
 1   date                 48892 non-null  datetime64[ns]
 2   direction            48892 non-null  object        
 3   internal             48892 non-null  object        
 4   operator_id          41491 non-null  Int64         
 5   is_missed_call       48892 non-null  bool          
 6   calls_count          48892 non-null  int64         
 7   call_duration        48892 non-null  int64         
 8   total_call_duration  48892 non-null  int64         
 9   tariff_plan          48892 non-null  object        
 10  date_start           48892 non-null  object        
 11  month                48892 non-null  int64         
dtypes: Int64(1), bool(1), datetime64[ns](1), int64(5), object(4)
memory usage: 4.6+ MB


Сделаем столбец с общем временем разговара в минутах

In [21]:
tl['minute']= tl['total_call_duration']/60
#tl['minute']= tl['minute'].round(1)
tl['minute']= tl['minute'].apply(np.ceil)


Разделим датасет по месяцам

In [22]:
tl_aug=tl[(tl['date'] >= '2019-08-01') & (tl['date'] <= '2019-08-31')]

In [23]:
tl_sep=tl[(tl['date'] >= '2019-09-01') & (tl['date'] <= '2019-09-30')]

In [24]:
tl_oct=tl[(tl['date'] >= '2019-10-01') & (tl['date'] <= '2019-10-31')]

In [25]:
tl_nov=tl[(tl['date'] >= '2019-11-01') & (tl['date'] <= '2019-11-30')]

In [26]:
tl_aut=tl[(tl['date'] >= '2019-09-01') & (tl['date'] <= '2019-11-30')]

### Выбросы и аномалии

#### Количество пропущенных звонков

In [27]:
tl.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration,tariff_plan,date_start,month,minute
0,166377,2019-08-04,in,False,<NA>,True,2,0,4,B,2019-08-01,8,1.0
1,166377,2019-08-05,out,True,880022,True,3,0,5,B,2019-08-01,8,1.0
2,166377,2019-08-05,out,True,880020,True,1,0,1,B,2019-08-01,8,1.0
3,166377,2019-08-05,out,True,880020,False,1,10,18,B,2019-08-01,8,1.0
4,166377,2019-08-05,out,False,880022,True,3,0,25,B,2019-08-01,8,1.0


Количество пропущенных звонков в каждый из месяцев

Количество клиентов, которые пришли в каждый месяц

In [28]:
tl_aut['month_start']=pd.to_datetime(tl_aut['date_start']).dt.month
tl_aut.groupby('month_start').agg({'user_id':'nunique'}).reset_index()

,month_start,user_id
0,8,99
1,9,96
2,10,108


In [29]:
i=tl_aut.query('is_missed_call == True')\
.groupby(['month'])\
.agg({'calls_count': 'sum'})\
.sort_values(by='calls_count', ascending= False).reset_index()
i

,month,calls_count
0,10,153145
1,11,142482
2,9,64337


In [30]:
fig = px.bar(i, x='month',y="calls_count", color='month',
             title="Количество пропущенных звонков")
fig.show()

In [31]:
def missed_calls(df):
        a=df.query('is_missed_call == True').groupby(['user_id']).agg({'calls_count': 'sum'}).sort_values(by='calls_count', ascending= False)
        fig = px.box(a, y="calls_count",
             title="Количество пропущенных звонков")
        fig.show()

За сентябрь

In [32]:
missed_calls(tl_sep)

За октябрь

In [33]:
missed_calls(tl_oct)

За ноябрь

In [34]:
missed_calls(tl_nov)

Заметно, что от месяца к месяцу количество пропущенных звонков растет, как и аномальное значение. Особая разница между сентябрем и остальными двумя месяцами. Октябрь и ноябрь по количеству пропущенных сопоставимы.

#### Количество минут

In [35]:
tl_aut.groupby('month').agg({'minute':'sum'}).reset_index().sort_values(by='minute', ascending= False)

,month,minute
2,11,359308.0
1,10,355664.0
0,9,201367.0


In [36]:
def min_count(df):
        a=df.groupby('user_id').agg({'minute':'sum', 'operator_id':'nunique'})\
        .reset_index().sort_values(by='minute', ascending= False)
        fig = px.box(a, y="minute",
             title="Количество потраченных минут")
        fig.show()

За осень

In [37]:
min_count(tl_aut)

За сентябрь

In [38]:
min_count(tl_sep)

За октябрь

In [39]:
min_count(tl_oct)

За ноябрь

In [40]:
min_count(tl_nov)

##### Количество звонков

In [41]:
tl_aut.groupby('month').agg({'operator_id':'nunique'})\
        .reset_index().sort_values(by='operator_id', ascending= False)

,month,operator_id
2,11,782
1,10,738
0,9,440


In [42]:
tl_aut.groupby('month').agg({'calls_count':'sum'})\
        .reset_index().sort_values(by='calls_count', ascending= False)

,month,calls_count
1,10,316215
2,11,306372
0,9,146543


В сентябре разговаривали меньше всего, октябрь и ноябрь примерно равны по количеству минут.

In [43]:
def call_count(df):
        a=df.groupby('user_id').agg({'calls_count':'sum'})\
        .reset_index().sort_values(by='calls_count', ascending= False)
        fig = px.box(a, y='calls_count',title="Количество звонков")
        fig.show()

За осень

In [44]:
call_count(tl_aut)

За сентябрь

In [45]:
call_count_sep = call_count(tl_sep)
call_count_sep

За октябрь

In [46]:
call_count_oct = call_count(tl_oct)
call_count_oct

За ноябрь

In [47]:
call_count_nov = call_count(tl_nov)
call_count_nov

Ноябрь все так же лидирует по медианным показателям, как и по аномальным.
Количество звонков будет одним из параметров определения активных клиентов. Для сентрября больше 90, для октября 100 и для ноября 110.

По всем параметрам заметна тенденция на повышение показателей. Можно предположить, что в августе и сентябре пришла основная часть активных клиентов, количество операторов также увеличилось, что поспобоствовало увелечению количества звонков, минут и пропущенных вызовов. Октябрь и ноябрь отличаются незначительно.

##### Отношение пропущенных звонков к общему их количеству

In [48]:
missing_calls_share=(tl_aut


 .groupby(['user_id','month'])
 .agg(total_calls = ('calls_count','sum'))
 .reset_index()

 .merge(tl_aut.query('is_missed_call == True').groupby(['user_id']).agg(missing_calls = ('calls_count','sum')).reset_index(),
        on= ['user_id'],how = 'left')

 .assign(missing_calls_share = lambda x: x['missing_calls'] / x['total_calls'] * 100)
)

In [49]:
fig = px.histogram(missing_calls_share, x="month", y="missing_calls_share",
             color='month', barmode='group',
             height=400,title='Доля пропущенных звонков по месяцам')
fig.show()

In [50]:
fig = px.pie(missing_calls_share, values='missing_calls_share', names='month', title='Доля пропущенных звонков по месяцам')
fig.show()

Определим еще 1 критерий для выявления активных клиентов - доля пропущенных звонков не более 30% от всех звонков.

##### Среднее количество минут на одного оператора

In [51]:
def operator_relation(df):
        i=df.groupby(['user_id','month']).agg({'minute':'sum','operator_id':'nunique'}).reset_index()
        i['min_per_operator'] = i['minute']/i['operator_id']
        i['min_per_operator']=i['min_per_operator'].round(2)
        i=i.sort_values(by='min_per_operator',ascending=False)
        print(i)
        fig = px.box(i, y="min_per_operator",
             title="Среднее время разговора оператора")
        #fig= px.scatter(i, x="min_per_operator", y="operator_id", color="month")
        fig.show()


In [52]:
operator_relation(tl_aut)

     user_id  month  minute  operator_id  min_per_operator
263   167077     10    10.0            0               inf
288   167114     11     1.0            0               inf
446   167545      9     6.0            0               inf
443   167543      9     2.0            0               inf
612   168090     10     2.0            0               inf
..       ...    ...     ...          ...               ...
642   168202     11     1.0            1               1.0
414   167474      9     1.0            1               1.0
468   167634      9     1.0            1               1.0
308   167162     11     1.0            1               1.0
467   167631     10     0.0            0               NaN

[707 rows x 5 columns]


Определим последний параметр активности клиентов: количество минут на одного оператора: 60 минут

#### Анализ в разрезе тарифов

In [53]:
graph=tl_aut.groupby(['tariff_plan','month']).agg({'operator_id':'nunique','calls_count':'sum', 'minute':'sum','user_id':'nunique'}).reset_index()
graph

,tariff_plan,month,operator_id,calls_count,minute,user_id
0,A,9,97,86878,111675.0,19
1,A,10,229,181659,189050.0,33
2,A,11,219,148091,172578.0,30
3,B,9,159,25940,40989.0,66
4,B,10,252,56335,76358.0,92
5,B,11,283,69514,93278.0,96
6,C,9,184,33725,48703.0,88
7,C,10,257,78221,90256.0,148
8,C,11,280,88767,93452.0,135


Количество операторов по месяцам в зависимости от тарифа

In [54]:
fig = px.bar(graph, x="operator_id", y="tariff_plan", color="month", barmode="group")
fig.show()

Количество звонков по месяцам в зависимости от тарифа

In [55]:
fig = px.bar(graph, x="calls_count", y="tariff_plan", color="month", barmode="group")
fig.show()

Количество минут по месяцам в зависимости от тарифа

In [56]:
fig = px.bar(graph, x="minute", y="tariff_plan", color="month", barmode="group")
fig.show()

Количество клиентов по месяцам в зависимости от тарифа

In [57]:
fig = px.bar(graph, x="user_id", y="tariff_plan", color="month", barmode="group")
fig.show()

In [58]:
fig = px.scatter_matrix(graph, dimensions=["tariff_plan", "minute", "operator_id", "calls_count",'user_id'], color="month")
fig.show()

В разрезе тарифов можно заметить, что в самом дорогом тарифе прибавилось незначительное количество клиентов (и в принципе в этом тарифе самое малое количество клиентов). Но зато у этих клиентов много операторов, поэтому в октябре произошел скачок по этому показателю и далее соответственно другие (количество звонков октябре выросло на 70% по сравнению с предыдущим месяцем).

В тарифе В все более равномерно, но также есть зависимость от количества клиентов и операторов - увеличилось количество звонков и потраченных минут.

В тарифе С больше всего клиентов. И хотя в октябре пришли 60 новых клиентов, количество операторов выросло не так сильно (но больше, чем в В тарифе), количество минут и разговоров также выросли более активно, чем в тарифе В.

## Исследовательский анализ

### Фильтрация

In [59]:
tl_sep.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration,tariff_plan,date_start,month,minute
107,166377,2019-09-02,in,False,880028,False,2,66,75,B,2019-08-01,9,2.0
108,166377,2019-09-02,in,False,880026,False,1,31,34,B,2019-08-01,9,1.0
109,166377,2019-09-02,out,False,880026,False,16,1853,1983,B,2019-08-01,9,34.0
110,166377,2019-09-02,out,False,880026,True,6,0,168,B,2019-08-01,9,3.0
111,166377,2019-09-02,out,False,880028,False,20,2529,2687,B,2019-08-01,9,45.0


Напишем функция для группировки данных и добавления новых столбцов с параметрами для будущей фильтрации

In [60]:
def group_df(df):
        i=df.groupby(['user_id','tariff_plan','internal','month'])\
        .agg({'calls_count':'sum','operator_id':'nunique','minute':'sum'})\
        .reset_index().sort_values(by='tariff_plan',ascending= False)


        i['min_per_operator'] = i['minute']/i['operator_id']
        i['min_per_operator']=i['min_per_operator'].round(2)
        return i

In [61]:
def group(df):
        df=(df.groupby(['user_id','tariff_plan','internal','month'])\
        .agg({'calls_count':'sum','operator_id':'nunique','minute':'sum'})\
        .reset_index().sort_values(by='tariff_plan',ascending= False)

        .merge(df.query('is_missed_call == True').groupby(['user_id','month']).agg(missing_calls = ('calls_count','sum')).reset_index(),
        on= ['user_id','month'],how = 'left')


        )
        df['min_per_operator'] = df['minute']/df['operator_id']
        df['min_per_operator']=df['min_per_operator'].round(2)
        df['missing_calls_share'] = df['missing_calls']/df['calls_count']*100
        df['missing_calls_share']= df['missing_calls_share'].round(2)
        return df

In [62]:
tl_sep = group(tl_sep)
tl_sep.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00
3,167079,C,False,9,13,1,13.0,12.0,13.00,92.31
4,167079,C,True,9,3,0,2.0,12.0,inf,400.00


In [63]:
tl_oct = group(tl_oct)
tl_oct.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share
0,167016,C,False,10,378,1,221.0,255.0,221.00,67.46
1,167906,C,False,10,20,2,16.0,20.0,8.00,100.00
2,167144,C,False,10,3,0,3.0,3.0,inf,100.00
3,167150,C,False,10,2626,6,3125.0,1024.0,520.83,38.99
4,167150,C,True,10,652,6,486.0,1024.0,81.00,157.06


In [64]:
tl_nov = group(tl_nov)
tl_nov.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share
0,168606,C,True,11,8,1,65.0,12.0,65.00,150.00
1,167188,C,True,11,29,4,29.0,63.0,7.25,217.24
2,167199,C,True,11,2,1,0.0,98.0,0.00,4900.00
3,167200,C,False,11,80,1,70.0,60.0,70.00,75.00
4,167235,C,False,11,1014,1,1316.0,245.0,1316.00,24.16


Делаем срез активных клиентов

In [65]:
tl_sep_active = tl_sep.query('missing_calls_share <=70 and min_per_operator >=60 and calls_count >=90').user_id.unique()
tl_sep = tl_sep.query('user_id in @tl_sep_active')

tl_oct_active = tl_oct.query('missing_calls_share <=70 and min_per_operator >=60 and calls_count >=100').user_id.unique()
tl_oct = tl_oct.query('user_id in @tl_oct_active')

tl_nov_active = tl_nov.query('missing_calls_share <=70 and min_per_operator >=60 and calls_count >=110').user_id.unique()
tl_nov = tl_nov.query('user_id in @tl_nov_active')

Собираем их в единый датасет

In [66]:
tl_fin = tl_sep.append(tl_oct)
tl_final = tl_fin.append(tl_nov)
tl_final

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91
...,...,...,...,...,...,...,...,...,...,...
369,166896,A,False,11,144,3,199.0,76.0,66.33,52.78
372,166901,A,False,11,257,5,383.0,119.0,76.60,46.30
373,168187,A,True,11,424,34,633.0,6480.0,18.62,1528.30
374,168187,A,False,11,14337,46,18601.0,6480.0,404.37,45.20


####  Анализ активных клиентов

In [67]:
graph= tl_final.groupby(['tariff_plan','month']).agg({'operator_id':'sum','calls_count':'sum','minute':'sum','user_id':'nunique'}).reset_index()
graph

,tariff_plan,month,operator_id,calls_count,minute,user_id
0,A,9,60,85378,109951.0,6
1,A,10,235,179590,186090.0,19
2,A,11,329,147292,171694.0,20
3,B,9,112,23226,38312.0,27
4,B,10,251,54505,74112.0,45
5,B,11,269,66496,90804.0,40
6,C,9,141,31695,46070.0,28
7,C,10,184,72426,84513.0,41
8,C,11,249,61927,77966.0,51


Количество операторов по месяцам в зависимости от тарифа среди активных клиентов

In [68]:
fig = px.bar(graph, x="operator_id", y="tariff_plan", color="month", barmode="group")
fig.show()

Количество звонков по месяцам в зависимости от тарифа среди активных клиентов

In [69]:
fig = px.bar(graph, x="calls_count", y="tariff_plan", color="month", barmode="group")
fig.show()

Количество минут по месяцам в зависимости от тарифа среди активных клиентов

In [70]:
fig = px.bar(graph, x="minute", y="tariff_plan", color="month", barmode="group")
fig.show()

Количество клиентов по месяцам в зависимости от тарифа среди активных клиентов

In [71]:
fig = px.bar(graph, x="user_id", y="tariff_plan", color="month", barmode="group")
fig.show()

In [72]:
fig = px.scatter_matrix(graph, dimensions=["tariff_plan", "minute", "operator_id", "calls_count",'user_id'], color="month")
fig.show()

Графики показывают примерно ту же зависимость, что и при анализе всех клиентов. Можно прийти к выводу, что активные клиенты играют определяющую роль по главным параметрам

### Поиск активных клиентов с неоптимальными тарифами

In [73]:
tl_final.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91


In [74]:
internal= tl_final.query('internal==True')
non_internal= tl_final.query('internal==False')
internal.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91
11,167052,C,True,9,1,1,0.0,80.0,0.00,8000.00
13,167125,C,True,9,67,8,33.0,4695.0,4.12,7007.46


Функция для внутренних звонков

In [75]:
def f_internal(row):
        val= 0
        if row['tariff_plan'] == 'A' and row['minute'] >2000:
            val = (row['minute']-2000) *0.1
        elif row['tariff_plan'] == 'B' and row['minute'] > 500:
            val = (row['minute']-500) *0.15
        elif row['tariff_plan'] == 'C':
            val = row['minute'] *0.3
        return val

tl_final['payment_1']= tl_final.apply(f_internal,axis=1)
tl_final

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share,payment_1
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87,127.8
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00,0.0
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00,1.5
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78,263.1
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91,80.4
...,...,...,...,...,...,...,...,...,...,...,...
369,166896,A,False,11,144,3,199.0,76.0,66.33,52.78,0.0
372,166901,A,False,11,257,5,383.0,119.0,76.60,46.30,0.0
373,168187,A,True,11,424,34,633.0,6480.0,18.62,1528.30,0.0
374,168187,A,False,11,14337,46,18601.0,6480.0,404.37,45.20,1660.1


In [76]:
non_internal.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78
9,167112,C,False,9,3119,2,2508.0,1417.0,1254.00,45.43
12,167125,C,False,9,7558,8,10480.0,4695.0,1310.00,62.12
16,167150,C,False,9,1667,6,1806.0,754.0,301.00,45.23


Напишем функцию для подсчета выручки от внешних звонков

In [77]:
def f_non_internal(row):
        val=0
        if row['tariff_plan'] == 'A':
            val = row['minute'] * 0.3
        elif row['tariff_plan'] == 'B':
            val = row['minute'] *0.5
        elif row['tariff_plan'] == 'C':
            val = row['minute'] * 0.7
        return val

tl_final['payment_2']= tl_final.apply(f_non_internal,axis=1)
tl_final.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share,payment_1,payment_2
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87,127.8,298.2
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00,0.0,0.0
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00,1.5,3.5
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78,263.1,613.9
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91,80.4,187.6


Далее необходимо добавить фиксировнную оплату за тариф и количество операторов.

In [78]:
def current_pay(row):
        val=0
        if row['tariff_plan'] == 'A':
            val = row['payment_1'] + row['payment_2']+4000+ (row['operator_id']*50)
        elif row['tariff_plan'] == 'B':
             val = row['payment_1'] +2000+row['payment_2']+ (row['operator_id']*150)
        elif row['tariff_plan'] == 'C':
            val = row['payment_1'] +1000+row['payment_2']+ (row['operator_id']*300)
        return val

tl_final['final_pay']= tl_final.apply(current_pay,axis=1)
tl_final.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share,payment_1,payment_2,final_pay
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87,127.8,298.2,3526.0
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00,0.0,0.0,1300.0
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00,1.5,3.5,1005.0
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78,263.1,613.9,3977.0
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91,80.4,187.6,3368.0


Функции для определения стоимости при других тарифах

In [79]:
def if_A(row):
            val=0
            val = row['payment_1'] + 4000 + (row['operator_id']*50)
            return val

tl_final['if_A']= tl_final.apply(if_A,axis=1)
tl_final.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share,payment_1,payment_2,final_pay,if_A
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87,127.8,298.2,3526.0,4477.8
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00,0.0,0.0,1300.0,4050.0
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00,1.5,3.5,1005.0,4001.5
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78,263.1,613.9,3977.0,4613.1
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91,80.4,187.6,3368.0,4430.4


In [80]:
def if_B(row):
            val=0
            val = row['payment_1'] + 2000+ (row['operator_id']*150)
            return val

tl_final['if_B']= tl_final.apply(if_B,axis=1)
tl_final.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share,payment_1,payment_2,final_pay,if_A,if_B
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87,127.8,298.2,3526.0,4477.8,3177.8
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00,0.0,0.0,1300.0,4050.0,2150.0
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00,1.5,3.5,1005.0,4001.5,2001.5
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78,263.1,613.9,3977.0,4613.1,3313.1
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91,80.4,187.6,3368.0,4430.4,3130.4


In [81]:
def if_C(row):
            val=0
            val = row['payment_1'] + 1000 + (row['operator_id']*300)
            return val

tl_final['if_C']= tl_final.apply(if_C,axis=1)
tl_final.head()

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share,payment_1,payment_2,final_pay,if_A,if_B,if_C
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87,127.8,298.2,3526.0,4477.8,3177.8,3227.8
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00,0.0,0.0,1300.0,4050.0,2150.0,1300.0
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00,1.5,3.5,1005.0,4001.5,2001.5,1001.5
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78,263.1,613.9,3977.0,4613.1,3313.1,3363.1
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91,80.4,187.6,3368.0,4430.4,3130.4,3180.4


### Поиск более выгодных тарифов

Функция для сравнения трех тарифов и выбор самого выгодного

In [82]:
def optim(row):
        val=0
        if row['final_pay'] > row['if_A'] and row['if_A'] < row['if_B'] :
            val = row['if_A']
        elif row['final_pay'] > row['if_B'] and row['if_B'] < row['if_C']:
             val = row['if_B']
        elif row['final_pay'] > row['if_C']:
            val = row['if_C']
        else: val = row['final_pay']
        return val

tl_final['optim_pay']= tl_final.apply(optim,axis=1)
tl_final

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share,payment_1,payment_2,final_pay,if_A,if_B,if_C,optim_pay
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87,127.8,298.2,3526.0,4477.8,3177.8,3227.8,3177.8
1,167112,C,True,9,1,1,0.0,1417.0,0.00,141700.00,0.0,0.0,1300.0,4050.0,2150.0,1300.0,1300.0
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00,1.5,3.5,1005.0,4001.5,2001.5,1001.5,1001.5
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78,263.1,613.9,3977.0,4613.1,3313.1,3363.1,3313.1
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91,80.4,187.6,3368.0,4430.4,3130.4,3180.4,3130.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,166896,A,False,11,144,3,199.0,76.0,66.33,52.78,0.0,59.7,4209.7,4150.0,2450.0,1900.0,1900.0
372,166901,A,False,11,257,5,383.0,119.0,76.60,46.30,0.0,114.9,4364.9,4250.0,2750.0,2500.0,2500.0
373,168187,A,True,11,424,34,633.0,6480.0,18.62,1528.30,0.0,189.9,5889.9,5700.0,7100.0,11200.0,5700.0
374,168187,A,False,11,14337,46,18601.0,6480.0,404.37,45.20,1660.1,5580.3,13540.4,7960.1,10560.1,16460.1,7960.1


Выделяем тех, кто мог бы поменять тариф и платить меньше

In [83]:
non_effective = tl_final.query('final_pay != optim_pay')
non_effective

,user_id,tariff_plan,internal,month,calls_count,operator_id,minute,missing_calls,min_per_operator,missing_calls_share,payment_1,payment_2,final_pay,if_A,if_B,if_C,optim_pay
0,167059,C,False,9,189,7,426.0,30.0,60.86,15.87,127.8,298.2,3526.0,4477.8,3177.8,3227.8,3177.8
2,167053,C,True,9,2,0,5.0,194.0,inf,9700.00,1.5,3.5,1005.0,4001.5,2001.5,1001.5,1001.5
7,167109,C,False,9,427,7,877.0,204.0,125.29,47.78,263.1,613.9,3977.0,4613.1,3313.1,3363.1,3313.1
8,167109,C,True,9,276,7,268.0,204.0,38.29,73.91,80.4,187.6,3368.0,4430.4,3130.4,3180.4,3130.4
9,167112,C,False,9,3119,2,2508.0,1417.0,1254.00,45.43,752.4,1755.6,4108.0,4852.4,3052.4,2352.4,2352.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,166896,A,False,11,144,3,199.0,76.0,66.33,52.78,0.0,59.7,4209.7,4150.0,2450.0,1900.0,1900.0
372,166901,A,False,11,257,5,383.0,119.0,76.60,46.30,0.0,114.9,4364.9,4250.0,2750.0,2500.0,2500.0
373,168187,A,True,11,424,34,633.0,6480.0,18.62,1528.30,0.0,189.9,5889.9,5700.0,7100.0,11200.0,5700.0
374,168187,A,False,11,14337,46,18601.0,6480.0,404.37,45.20,1660.1,5580.3,13540.4,7960.1,10560.1,16460.1,7960.1


### Расчет выручки с изменением тарифов у активных клиентов на оптимальные

In [84]:
active_current_income = non_effective['final_pay'].sum()
active_current_income

1660834.4

Сколько платят активные клиенты сейчас

In [118]:
active_optim_income = non_effective['optim_pay'].sum()
active_optim_income.round(2)

1031415.6

Сколько бы платили при смене тарифов

In [86]:
income = pd.DataFrame([['active_current_income', 1660834], \
                   ['active_optim_income', 1031415]], columns=['type', 'amount'])
income

,type,amount
0,active_current_income,1660834
1,active_optim_income,1031415


#### График изменения выручки по активным клиентам

In [119]:
fig = px.bar(income, x='amount', y="type", color="type", title="Изменение выручки среди активных клиентов")
fig.show()

Почти на 40% снизится выручка среди активных клиентов при смене на оптимальные тарифы

### Расчет общей выручки по всем клиентам

In [88]:
tl_aut=(tl_aut
 .groupby(['user_id','tariff_plan','month'])
 .agg(total_operators = ('operator_id','nunique'), total_calls = ('calls_count','sum'))
 .reset_index()

# считаем количество пропущенных
 .merge(tl_aut.query('is_missed_call == True').groupby(['user_id','tariff_plan','month']).agg(missing_calls = ('calls_count','sum')).reset_index(),
        on = ['user_id','tariff_plan','month'],how = 'left')
# подсчитаем долю пропущенных звонков:
 .assign(missing_calls_share = lambda x: x['missing_calls'] / x['total_calls'] * 100)

# добавляем исходящие внешние (длительность в секундах):
 .merge(tl_aut.query('direction == "out" and internal == False').groupby(['user_id','tariff_plan','month']).agg(ext_out_duration = ('total_call_duration','sum')).reset_index(),
        on = ['user_id','tariff_plan','month'],how = 'left')
# добавляем исходящие внутренние (длительность в секундах):
 .merge(tl_aut.query('direction == "out" and internal == True').groupby(['user_id','tariff_plan','month']).agg(int_out_duration = ('total_call_duration','sum')).reset_index(),
        on = ['user_id','tariff_plan','month'],how = 'left')
 )
tl_aut['ext_out_duration']=tl_aut['ext_out_duration']/60
tl_aut['int_out_duration']=tl_aut['int_out_duration']/60
tl_aut.head()

,user_id,tariff_plan,month,total_operators,total_calls,missing_calls,missing_calls_share,ext_out_duration,int_out_duration
0,166377,B,9,4,1452,556.0,38.292011,1763.916667,0.150000
1,166377,B,10,5,1406,467.0,33.214794,1653.450000,0.083333
2,166377,B,11,4,1314,466.0,35.464231,1589.733333,2.116667
3,166391,C,9,1,4,3.0,75.000000,NaN,NaN
4,166391,C,10,0,2,2.0,100.000000,NaN,NaN


Функция для вычисления оплаты сверх лимита по бесплатным минутам для звонков внутри сети.

In [89]:
def f_internal(row):
        val= 0
        if row['tariff_plan'] == 'A' and row['int_out_duration'] >2000:
            val= (row['int_out_duration']-2000) * 0.1
        elif row['tariff_plan'] == 'B' and row['int_out_duration'] > 500:
            val = (row['int_out_duration']-500) *0.15
        elif row['tariff_plan'] == 'C':
            val = row['int_out_duration'] *0.3
        return val

tl_aut['payment_1']= tl_aut.apply(f_internal,axis=1)
tl_aut.sort_values(by='int_out_duration',ascending= False)

,user_id,tariff_plan,month,total_operators,total_calls,missing_calls,missing_calls_share,ext_out_duration,int_out_duration,payment_1
427,167497,A,11,17,2282,714.0,31.288344,1852.350000,1392.816667,0.0000
188,166916,A,11,14,3025,743.0,24.561983,1436.566667,1334.366667,0.0000
614,168091,B,11,16,3493,1117.0,31.978242,3939.450000,658.916667,23.8375
607,168062,A,11,27,25332,14963.0,59.067583,16461.366667,523.816667,0.0000
382,167359,B,11,8,809,232.0,28.677379,395.733333,406.800000,0.0000
...,...,...,...,...,...,...,...,...,...,...
700,168579,B,11,2,172,133.0,77.325581,NaN,NaN,0.0000
701,168583,B,10,1,1,NaN,NaN,NaN,NaN,0.0000
702,168583,B,11,2,39,18.0,46.153846,NaN,NaN,0.0000
703,168598,C,11,1,101,80.0,79.207921,NaN,NaN,NaN


Расчет внешних звонков

In [90]:
def f_non_internal(row):
        val=0
        if row['tariff_plan'] == 'A':
            val = row['ext_out_duration'] * 0.3
        elif row['tariff_plan'] == 'B':
            val = row['ext_out_duration'] *0.5
        elif row['tariff_plan'] == 'C':
            val = row['ext_out_duration'] * 0.7
        return val

tl_aut['payment_2']= tl_aut.apply(f_non_internal,axis=1)
tl_aut.head()

,user_id,tariff_plan,month,total_operators,total_calls,missing_calls,missing_calls_share,ext_out_duration,int_out_duration,payment_1,payment_2
0,166377,B,9,4,1452,556.0,38.292011,1763.916667,0.150000,0.0,881.958333
1,166377,B,10,5,1406,467.0,33.214794,1653.450000,0.083333,0.0,826.725000
2,166377,B,11,4,1314,466.0,35.464231,1589.733333,2.116667,0.0,794.866667
3,166391,C,9,1,4,3.0,75.000000,NaN,NaN,NaN,NaN
4,166391,C,10,0,2,2.0,100.000000,NaN,NaN,NaN,NaN


Соединяем внешние и внутренние звонки

In [91]:

tl_aut['payment_1']= tl_aut['payment_1'].fillna(0)
tl_aut['payment_2'] =tl_aut['payment_2'].fillna(0)
tl_aut.head()

,user_id,tariff_plan,month,total_operators,total_calls,missing_calls,missing_calls_share,ext_out_duration,int_out_duration,payment_1,payment_2
0,166377,B,9,4,1452,556.0,38.292011,1763.916667,0.150000,0.0,881.958333
1,166377,B,10,5,1406,467.0,33.214794,1653.450000,0.083333,0.0,826.725000
2,166377,B,11,4,1314,466.0,35.464231,1589.733333,2.116667,0.0,794.866667
3,166391,C,9,1,4,3.0,75.000000,NaN,NaN,0.0,0.000000
4,166391,C,10,0,2,2.0,100.000000,NaN,NaN,0.0,0.000000


Вычисляем текущую выручку, добавляя фиксированную оплату и оплату за операторов.

In [92]:
def current_pay(row):
        val=0
        if row['tariff_plan'] == 'A':
            val = row['payment_1'] +row['payment_2']+4000+ (row['total_operators']*50)
        elif row['tariff_plan'] == 'B':
             val = row['payment_1'] +row['payment_2']+ 2000+ (row['total_operators']*150)
        elif row['tariff_plan'] == 'C':
            val = row['payment_1'] +row['payment_2'] + 1000+ (row['total_operators']*300)
        return val

tl_aut['final_pay']= tl_aut.apply(current_pay,axis=1)
general_current_income = tl_aut['final_pay'].sum().round(1)
print(general_current_income)
tl_aut

1847037.3


,user_id,tariff_plan,month,total_operators,total_calls,missing_calls,missing_calls_share,ext_out_duration,int_out_duration,payment_1,payment_2,final_pay
0,166377,B,9,4,1452,556.0,38.292011,1763.916667,0.150000,0.00,881.958333,3481.958333
1,166377,B,10,5,1406,467.0,33.214794,1653.450000,0.083333,0.00,826.725000,3576.725000
2,166377,B,11,4,1314,466.0,35.464231,1589.733333,2.116667,0.00,794.866667,3394.866667
3,166391,C,9,1,4,3.0,75.000000,NaN,NaN,0.00,0.000000,1300.000000
4,166391,C,10,0,2,2.0,100.000000,NaN,NaN,0.00,0.000000,1000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
702,168583,B,11,2,39,18.0,46.153846,NaN,NaN,0.00,0.000000,2300.000000
703,168598,C,11,1,101,80.0,79.207921,NaN,NaN,0.00,0.000000,1300.000000
704,168601,C,11,2,692,273.0,39.450867,553.200000,0.000000,0.00,387.240000,1987.240000
705,168603,B,11,1,15,5.0,33.333333,19.783333,NaN,0.00,9.891667,2159.891667


In [93]:
avg= tl_aut.groupby(['tariff_plan','month']).agg({'final_pay':'sum','user_id':'nunique'}).reset_index()
avg['average_payment'] = avg['final_pay']/avg['user_id']
avg

,tariff_plan,month,final_pay,user_id,average_payment
0,A,9,112608.225000,19,5926.748684
1,A,10,193384.830000,33,5860.146364
2,A,11,168033.650000,30,5601.121667
3,B,9,170003.933333,66,2575.817172
4,B,10,245397.508333,92,2667.364221
5,B,11,262269.837500,96,2731.977474
6,C,9,168667.961667,88,1916.681383
7,C,10,268186.655000,148,1812.071993
8,C,11,258484.723333,135,1914.701654


In [94]:
fig = px.bar(avg, x='tariff_plan', y="average_payment", color="month", title="Изменение среднего чека по тарифам")
fig.show()

#### Доля выручки по тарифам

In [95]:
tariff_share= tl_aut.groupby(['tariff_plan']).agg({'final_pay':'sum'}).reset_index()
tariff_share

,tariff_plan,final_pay
0,A,474026.705000
1,B,677671.279167
2,C,695339.340000


In [96]:
fig = px.pie(tariff_share, values='final_pay', names='tariff_plan', title='Доля выручки по тарифам')
fig.show()

In [97]:
general_current_income

1847037.3

In [98]:
active_current_income= active_current_income.round(1)
active_current_income

1660834.4

In [99]:
active_optim_income = active_optim_income.round(1)
active_optim_income

1031415.6

In [100]:
general_optim_income = general_current_income - active_current_income +active_optim_income
general_optim_income

1217618.5

In [101]:
general_income = pd.DataFrame([['general_current_income', 1847037], ['general_optim_income',1217618]], \
                              columns=['type', 'amount'])
general_income

,type,amount
0,general_current_income,1847037
1,general_optim_income,1217618


#### График изменения выручки по всем клиентам

In [102]:
fig = px.bar(general_income, x='amount', y="type", color="type", title="Изменение выручки")
fig.show()

### Вывод

Общая выручка со всех клиентов при условии оптимальных тарифов для активных клиентов снизится  на 630 тысяч.
Среди активных клиентов разница в 38%.
Во временном промежутке прослеживается тенденция увеличения показателей (количество звонков, среднее время разговора).

## Проверка гипотез

### Гипотеза 1: А0) Выручка от активных клиентов осталась на прежнем уровне после смены тарифов  А1) Выручка от активных клиентов изменилась после смены тарифов на более выгодные

In [103]:
non_effective.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444 entries, 0 to 375
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              444 non-null    int64  
 1   tariff_plan          444 non-null    object 
 2   internal             444 non-null    bool   
 3   month                444 non-null    int64  
 4   calls_count          444 non-null    int64  
 5   operator_id          444 non-null    int64  
 6   minute               444 non-null    float64
 7   missing_calls        444 non-null    float64
 8   min_per_operator     444 non-null    float64
 9   missing_calls_share  444 non-null    float64
 10  payment_1            444 non-null    float64
 11  payment_2            444 non-null    float64
 12  final_pay            444 non-null    float64
 13  if_A                 444 non-null    float64
 14  if_B                 444 non-null    float64
 15  if_C                 444 non-null    flo

In [104]:
fig = px.box(non_effective, y="final_pay",
             title="Текущая выручка")
fig.show()

In [105]:
non_effective['optim_pay'].describe()

count      444.000000
mean      2323.008108
std       1425.241804
min       1001.200000
25%       1463.425000
50%       1900.000000
75%       2595.587500
max      12380.000000
Name: optim_pay, dtype: float64

In [106]:
fig = px.box(non_effective, y="optim_pay",
             title="Оптимальная выручка")
fig.show()

Выбросы есть, но они одни и те же в двух выборках.

In [107]:
i=non_effective.groupby(['tariff_plan']).agg({'final_pay':'sum','optim_pay':'sum'}).reset_index()
i

,tariff_plan,final_pay,optim_pay
0,A,512501.6,237131.1
1,B,587083.8,383469.8
2,C,561249.0,410814.7


In [108]:
i['final_pay'].describe()

count         3.000000
mean     553611.466667
std       37873.144519
min      512501.600000
25%      536875.300000
50%      561249.000000
75%      574166.400000
max      587083.800000
Name: final_pay, dtype: float64

In [109]:
i['optim_pay'].describe()

count         3.000000
mean     343805.200000
std       93388.749842
min      237131.100000
25%      310300.450000
50%      383469.800000
75%      397142.250000
max      410814.700000
Name: optim_pay, dtype: float64

Посмотрев на средние значения уже можно сказать, что нулевая гипотеза будет отвергнута,поэтому можно установить более строгий уровень стат. значимости. В обеих выборках по 144 клиента.

In [110]:
before = non_effective['final_pay']
after = non_effective['optim_pay']
alpha = 0.01 # уровень статистической значимости

results = st.ttest_rel(before, after)


print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: 2.292224641978561e-28
Отвергаем нулевую гипотезу


### Гипотеза 2: А0)  Клиенты более дешевого тарифа разговаривают столько же времени, как и клиенты более дорогого тарифа А1) Клиенты более дешевого тарифа  разговаривают больше или меньше, чем клиенты более дорогого тарифа

In [111]:
tl.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration,tariff_plan,date_start,month,minute
0,166377,2019-08-04,in,False,<NA>,True,2,0,4,B,2019-08-01,8,1.0
1,166377,2019-08-05,out,True,880022,True,3,0,5,B,2019-08-01,8,1.0
2,166377,2019-08-05,out,True,880020,True,1,0,1,B,2019-08-01,8,1.0
3,166377,2019-08-05,out,True,880020,False,1,10,18,B,2019-08-01,8,1.0
4,166377,2019-08-05,out,False,880022,True,3,0,25,B,2019-08-01,8,1.0


In [112]:
tl_grouped = tl.groupby(['user_id','tariff_plan']).agg({'minute': 'sum'}).reset_index()
tl_grouped

,user_id,tariff_plan,minute
0,166377,B,6728.0
1,166391,C,31.0
2,166392,C,400.0
3,166399,C,23.0
4,166405,B,24702.0
...,...,...,...
302,168583,B,42.0
303,168598,C,94.0
304,168601,C,750.0
305,168603,B,24.0


In [113]:
fig = px.box(tl_grouped, y="minute",
             title="Количество минут")
fig.show()

In [114]:
tl.groupby('tariff_plan').agg({'minute':'sum','user_id':'nunique'}).reset_index()

,tariff_plan,minute,user_id
0,A,505862.0,34
1,B,228440.0,112
2,C,235311.0,161


По таблице видно,что самым дорогим тарифом пользуется только 34 клиента, но количество минут более чем в 2 раза больше, чем у двух других тарифов. Сравнивая тариф А, необходимо учитывать критерий Стьюдента из-за больших различий. Тарифы В и С сопоставимы по количеству минут, данную поправку можно не использовать.

In [115]:
C = tl_grouped[tl_grouped['tariff_plan'] =='C']['minute']
B = tl_grouped[tl_grouped['tariff_plan'] =='B']['minute']
results = st.ttest_ind(C , B)

alpha =0.01

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: 0.2645554361884787
Не получилось отвергнуть нулевую гипотезу


In [116]:
C = tl_grouped[tl_grouped['tariff_plan'] =='C']['minute']
A = tl_grouped[tl_grouped['tariff_plan'] =='A']['minute']
results = st.ttest_ind(C , A, equal_var = False)

alpha =0.01

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: 0.051127705739898985
Не получилось отвергнуть нулевую гипотезу


In [117]:
A = tl_grouped[tl_grouped['tariff_plan'] =='A']['minute']
B = tl_grouped[tl_grouped['tariff_plan'] =='B']['minute']
results = st.ttest_ind(A , B, equal_var = False)

alpha =0.01

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: 0.061763112288472644
Не получилось отвергнуть нулевую гипотезу


### Вывод

Как и предполагалось, выручка после смены тарифов изменилась. В то же время мы выяснили, что от тарифа не зависит продолжительность разговора.

## Итоговый вывод

Подводя итог, можно утверждать, что выручка определенно снизится при смене тарифов. Это может поспособствовать увелечению лояльности со стороны клиента, но надо быть готовым к потерям в доходах.
Если моделировать такую ситуацию, при которой активные клиенты будут использовать оптимальные тарифы, то выручка упадет на 38%.
Среди всех клиентов такая модель показывает, что доходы упадут на 630 тысяч.

Распределение доходов по тарифам получилось такое: по 39 и 37% тарифы `А`и `C` соответственно, а тариф `В`-24%
В разрезе тарифов среди всех клиентов  также были сделаны следующие выводы:
+ Общее повышение активности наблюдается в октябре. В это время повысилось количество разговоров, пропущенных звонков, количество клиентов и операторов, причем больше всего это заметно на тарифе А (количество операторов с 97 до 229 и количество звонков с 86,5 тысяч до 181,5 тысяч звонков)
+ Доля пропущенных звонков в сентябре -540 тысяч и снизилась до 240 тысяч в октябре, в ноябре 190 тысяч.
+ Среди активных клиентов картина похожая, что значит весомость активных клиентов в определении параметров.

Активных клиентов определяли по трем критериям: доля пропущенных звонков меньше 70%, оператор в среднем разговаривает час и количество звонков в среднем более 100.

Проверили главную гипотезу о том, что выручка упадёт после перевода клиентов на более выгодные тарифы- она подтвердилась. Это прослеживалось и по средним значениям выручки.
Также, еще одной гипотезой было то, что клиенты дешевого тарифа разговаривают столько же, сколько и клиенты дорогих тарифов. Так как выборки могли быть неравноправными, то использовался параметр Стьюдента для более точного теста, уровень стат. значимости во всех тестах был 0.01. Гипотеза оказалась верной - вне зависимости от тарифа, клиенты разговаривают одинаково.

## Рекомендации
### При снижении выручки после оптимизации тарифных планов, можно проанализировать соотношение внутренних и внешних звонков: если внутренних 65-70%, то снизить лимит минут; Если клиент - крупный колл-центр, где более 200 сотрудников, то повысить плату за 1 оператора путем балансировки существующих тарифов или добавлением промежуточного.